## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
dli_dict = {}        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
nomes_anp = []      # Conterá os nomes ANP dos poços
charId_inicio, charId_fim = 7, 10   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    
    nome = leitura.origins[0].well_name

    nomes_anp.append(nome)    # armazenando o nome do poço
    
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    
    nome_abreviado = nome[charId_inicio : charId_fim]   # identificador do poço, e.g. '900'
    
    # Armazenando o arquivo lógico como valor em um dicionário onde a chave é o identificador do poço
    dli_dict[nome_abreviado] = leitura   

In [25]:
dli_dict

{'-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-S': LogicalFile(GEOLOAD.1),
 '595': LogicalFile(GEOLOAD.1),
 '605': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '659': LogicalFile(GEOLOAD.1),
 '900': LogicalFile(GEOLOAD.1),
 '889': LogicalFile(GEOLOAD.1),
 '912': LogicalFile(GEOLOAD.1),
 '897': LogicalFile(GEOLOAD.1),
 '910': LogicalFile(GEOLOAD.1),
 '907': LogicalFile(GEOLOAD.1)}

In [32]:
dli_dict['605'].frames[0].channels

[Channel(TDEP),
 Channel(BS),
 Channel(TENS),
 Channel(SP),
 Channel(GR),
 Channel(NPHI),
 Channel(TIME),
 Channel(IHV),
 Channel(ICV),
 Channel(DT)]

## Cria dataframes para os poços

In [44]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['GR', 'BS', 'NPHI', 'CALI', 'RHOB', 'DRHO', 'PE', 'HDRS', 'HMRS', 'DTC', 'BSZ']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = pd.DataFrame(curvas[curvas_utilizadas])
    except:
        pass

In [45]:
dlis_df_dict['605']

KeyError: '605'

In [ ]:
# Transformados os valores -999.25 em nulos
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Adiciona coluna DCALI

In [ ]:
add_DCALI(dlis_df_dict)

limita_curva(dlis_df_dict, "DCALI", -2, 2)

## Remove valores DRHO indesejados

In [ ]:
limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)